In [2]:
import requests, json, time, math
import pandas as pd
from tqdm import tqdm_notebook as tqdm  ##address the new line problem of tqdm in jupyter notebook
from sqlalchemy import *
import pymysql

import users id

In [29]:
df=pd.read_csv('/Users/liuxin/Documents/data_lab/W1/steam_user_id.csv',header=None)  ##不让第一行读为column name
df.head()

,0
0,76561198158086086
1,76561198074188133
2,76561198058088990
3,76561198175177483
4,76561198042649112


obtain url, API key from steam website

In [23]:
base_url= 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
dic_temp = {}

In [30]:
try:
    with tqdm(range(df.shape[0])) as t:
        for i in t:
            user_id=str((df.loc[i][0]))
            param={
           'key': '77551F6A63EFCF3C03A15066DFB9FB6F',
           'steamid':user_id,
           'format' : 'json' }
            r = requests.get(base_url, params = param, headers = {})
            user_inventory = r.json().get('response').get('games')
            time.sleep(.5)
            dic_temp.update({user_id:user_inventory})
except KeyboardInterrupt:
    t.close()
    raise
t.close()

write user_id and the corresponding game information into a csv file.

In [37]:
with open('/Users/liuxin/Documents/data_lab/recommendation_system/user_inventory.csv', 'w') as f:
    for user_id, user_inventory in list(dic_temp.items()):
        f.write(json.dumps({str(user_id):user_inventory}))
        f.write('\n')

get application information

In [15]:
url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
r = requests.get(url)

In [16]:
dic_app_list = r.json()

In [17]:
lst_app_id = [i.get('appid') for i in dic_app_list.get('applist').get('apps')]
len(lst_app_id)


90796

In [ ]:
current_count = 0
path_app_detail_sample = '/Users/liuxin/Documents/data_lab/recommendation_system/app_detail.csv'
with open(path_app_detail_sample, 'w') as f:
    for app_id in tqdm(lst_app_id[0:50]):
        url_app_detail = ('http://store.steampowered.com/api/appdetails?appids=%s') % (app_id)
        for i in range(3):
            try:
                r = requests.get(url_app_detail)
                result = r.json()
                break
            except Exception as e:
                print(e)
                time.sleep(5)
        f.write(json.dumps(result))
        f.write('\n')

        if current_count > 0 and current_count % 200 == 0:
            time.sleep(300)
        else:
            time.sleep(.5)

In [ ]:
path_app_detail = '/Users/liuxin/Documents/data_lab/recommendation_system/app_detail.csv'

with open(path_app_detail, 'r') as f:
    
    dic_steam_app = {
        'initial_price':{},
        'name':{},
        'score':{},
        'windows':{},
        'mac':{},
        'linux':{},
        'type':{},
        'release_date':{},
        'recommendation':{},
        'header_image':{}
    }
    lst_raw_string = f.readlines()[:14]

    for raw_string in tqdm(lst_raw_string):
        try:
            app_data = list(json.loads(raw_string).values())[0]
            if app_data.get('success'):
                app_data = app_data.get('data')
                steam_id = app_data.get('steam_appid')
                initial_price = app_data.get('price_overview',{}).get('initial')
                if app_data.get('is_free') == True:
                    initial_price = 0
                app_name = app_data.get('name')
                critic_score = app_data.get('metacritic', {}).get('score')
                app_type = app_data.get('type')
                for (platform, is_supported) in app_data.get('platforms',{}).items():
                    if is_supported == True:
                        dic_steam_app[platform].update({steam_id:1})
                    else:
                        dic_steam_app[platform].update({steam_id:0})
                if app_data.get('release_date',{}).get('coming_soon') == False:
                    release_date = app_data.get('release_date',{}).get('date')
                    if not release_date == '':
                        try:
                            release_date = datetime.strptime(release_date, '%b %d, %Y')
                        except Exception as e:
                            try:
                                release_date = datetime.strptime(release_date, '%d %b, %Y')
                            except:
                                release_date = None
                        

                recommendation = app_data.get('recommendations',{}).get('total')
                header_image = app_data.get('header_image')
                dic_steam_app['initial_price'].update({steam_id:initial_price})
                dic_steam_app['name'].update({steam_id:app_name})
                dic_steam_app['score'].update({steam_id:critic_score})
                dic_steam_app['type'].update({steam_id:app_type})
                dic_steam_app['release_date'].update({steam_id:release_date})
                dic_steam_app['recommendation'].update({steam_id:recommendation})
                dic_steam_app['header_image'].update({steam_id:header_image})
            time.sleep(.1)
        except:
            pass



work with MySQL in Python

In [21]:
df_app_info = pd.DataFrame(dic_steam_app)

In [22]:
df_app_info.index.name = 'app_id'   ##the first col is always the index

In [23]:
df_app_info.reset_index(inplace=True)
df_app_info.head()

,app_id,initial_price,name,score,windows,mac,linux,type,release_date,recommendation,header_image
0,535530,1999,Demon Hunter,None,1,0,0,game,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...
1,535540,799,TANE DLC: Laadgs Transporter,None,1,1,0,dlc,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...
2,535580,899,TANE DLC: Avmz Intercity 71,None,1,1,0,dlc,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...
3,535600,799,TANE DLC: Hccrrs Car Transporter,None,1,1,0,dlc,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...
4,535650,299,Dwingle : B.O.T,None,1,0,0,game,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...


In [25]:
user = 'root'
password = 'x9Liu12@'
host = 'localhost'
db_name = 'steam'
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{db_name}?charset=utf8mb4')

In [26]:
df_app_info.to_sql('tbl_app_info', engine, if_exists='replace',index=False)

In [27]:
pd.read_sql_query('''
    select * from tbl_app_info 
''', engine)

,app_id,initial_price,name,score,windows,mac,linux,type,release_date,recommendation,header_image
0,535530,1999,Demon Hunter,None,1,0,0,game,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...
1,535540,799,TANE DLC: Laadgs Transporter,None,1,1,0,dlc,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...
2,535580,899,TANE DLC: Avmz Intercity 71,None,1,1,0,dlc,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...
3,535600,799,TANE DLC: Hccrrs Car Transporter,None,1,1,0,dlc,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...
4,535650,299,Dwingle : B.O.T,None,1,0,0,game,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...
5,535660,0,Space Ribbon Demo,None,1,1,1,demo,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...
6,535680,499,Moorhuhn schlägt zurück,None,1,0,0,game,None,None,https://steamcdn-a.akamaihd.net/steam/apps/535...
